# WMI607: AI and Machine Learning Take-Home Exam
## Section C: Feature Engineering & Representation (Question 3)

**Objective**: Apply and justify feature engineering techniques with domain knowledge.

---

In [21]:
# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
from sklearn.decomposition import PCA
from sklearn.feature_selection import SelectKBest, f_classif, mutual_info_classif
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, cross_val_score
import json
import warnings
warnings.filterwarnings('ignore')

plt.style.use('seaborn-v0_8-whitegrid')
print("Libraries imported successfully!")

Libraries imported successfully!


In [22]:
# Load preprocessed data
import os

# Check if preprocessed data exists, otherwise load from Kaggle
if os.path.exists('data/preprocessed_data.pkl'):
    df = pd.read_pickle('data/preprocessed_data.pkl')
    with open('data/metadata.json', 'r') as f:
        metadata = json.load(f)
    target_col = metadata['target_column']
    print(f"Loaded preprocessed data: {df.shape[0]:,} rows, {df.shape[1]} columns")
else:
    import kagglehub
    path = kagglehub.dataset_download("ziya07/iiot-edge-computing-dataset")
    csv_files = [f for f in os.listdir(path) if f.endswith('.csv')]
    df = pd.read_csv(os.path.join(path, csv_files[0]))
    # Identify target column
    potential_targets = [col for col in df.columns if any(t in col.lower() for t in ['target', 'label', 'class', 'anomaly', 'attack'])]
    target_col = potential_targets[0] if potential_targets else df.columns[-1]
    print(f"Loaded data from Kaggle: {df.shape[0]:,} rows, {df.shape[1]} columns")

print(f"Target column: {target_col}")

JSONDecodeError: Expecting value: line 37 column 24 (char 756)

## 1. Feature Analysis and Domain Knowledge

### Domain Context: IIoT Edge Computing & Infrastructure Monitoring

The dataset contains sensor readings from Industrial IoT edge computing devices. Key domain insights:

1. **Sensor readings** often have drift and calibration issues
2. **Temporal patterns** are crucial for anomaly detection
3. **Cross-sensor correlations** reveal system state
4. **Statistical features** capture distribution changes

In [ ]:
# Separate features and target
X = df.drop(columns=[target_col])
y = df[target_col]

# Identify numeric and categorical columns
numeric_cols = X.select_dtypes(include=[np.number]).columns.tolist()
categorical_cols = X.select_dtypes(include=['object', 'category']).columns.tolist()

print(f"\nFeature Summary:")
print(f"  • Numeric features: {len(numeric_cols)}")
print(f"  • Categorical features: {len(categorical_cols)}")
print(f"  • Target classes: {y.nunique()}")

---

## 2. Feature Representation Technique 1: Statistical Feature Engineering

**Justification**: IIoT sensor data exhibits statistical patterns that raw values don't capture. Creating statistical features helps models understand data distribution characteristics.

In [ ]:
# Feature Representation 1: Statistical Features
print("=" * 60)
print("FEATURE REPRESENTATION 1: STATISTICAL ENGINEERING")
print("=" * 60)

X_numeric = X[numeric_cols].copy()

# Handle any remaining missing values
X_numeric = X_numeric.fillna(X_numeric.median())

# Create statistical features
stat_features = pd.DataFrame(index=X_numeric.index)

# Row-wise statistics (for each sample across all features)
stat_features['row_mean'] = X_numeric.mean(axis=1)
stat_features['row_std'] = X_numeric.std(axis=1)
stat_features['row_min'] = X_numeric.min(axis=1)
stat_features['row_max'] = X_numeric.max(axis=1)
stat_features['row_range'] = stat_features['row_max'] - stat_features['row_min']
stat_features['row_skew'] = X_numeric.skew(axis=1)
stat_features['row_kurtosis'] = X_numeric.kurtosis(axis=1)

# Percentile-based features
stat_features['row_q25'] = X_numeric.quantile(0.25, axis=1)
stat_features['row_q75'] = X_numeric.quantile(0.75, axis=1)
stat_features['row_iqr'] = stat_features['row_q75'] - stat_features['row_q25']

# Combine with original features
X_stat = pd.concat([X_numeric, stat_features], axis=1)

print(f"\n📊 Statistical Features Created:")
print(f"   • Original features: {len(numeric_cols)}")
print(f"   • New statistical features: {len(stat_features.columns)}")
print(f"   • Total features: {X_stat.shape[1]}")
print(f"\n   Features added: {list(stat_features.columns)}")

In [ ]:
# Evaluate impact of statistical features
from sklearn.model_selection import cross_val_score

# Baseline model (original features only)
scaler = StandardScaler()
X_orig_scaled = scaler.fit_transform(X_numeric)

rf_baseline = RandomForestClassifier(n_estimators=50, random_state=42, n_jobs=-1)
scores_baseline = cross_val_score(rf_baseline, X_orig_scaled, y, cv=5, scoring='accuracy')

# Model with statistical features
X_stat_scaled = scaler.fit_transform(X_stat)
rf_stat = RandomForestClassifier(n_estimators=50, random_state=42, n_jobs=-1)
scores_stat = cross_val_score(rf_stat, X_stat_scaled, y, cv=5, scoring='accuracy')

print(f"\n📈 PERFORMANCE COMPARISON:")
print(f"   Baseline (original features): {scores_baseline.mean():.4f} (+/- {scores_baseline.std()*2:.4f})")
print(f"   With statistical features:    {scores_stat.mean():.4f} (+/- {scores_stat.std()*2:.4f})")
print(f"   Improvement: {(scores_stat.mean() - scores_baseline.mean())*100:.2f}%")

---

## 3. Feature Representation Technique 2: Dimensionality Reduction with PCA

**Justification**: High-dimensional IIoT data often contains redundant information. PCA extracts principal components that capture maximum variance while reducing dimensionality for faster training.

In [ ]:
# Feature Representation 2: PCA-based Representation
print("=" * 60)
print("FEATURE REPRESENTATION 2: PCA DIMENSIONALITY REDUCTION")
print("=" * 60)

# Scale data for PCA
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_numeric)

# Apply PCA
pca_full = PCA()
pca_full.fit(X_scaled)

# Calculate cumulative explained variance
cumsum = np.cumsum(pca_full.explained_variance_ratio_)

# Find number of components for 95% variance
n_components_95 = np.argmax(cumsum >= 0.95) + 1
n_components_90 = np.argmax(cumsum >= 0.90) + 1

print(f"\n📊 PCA ANALYSIS:")
print(f"   • Original dimensions: {X_scaled.shape[1]}")
print(f"   • Components for 90% variance: {n_components_90}")
print(f"   • Components for 95% variance: {n_components_95}")
print(f"   • Dimensionality reduction: {(1 - n_components_95/X_scaled.shape[1])*100:.1f}%")

In [ ]:
# Visualize PCA variance
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Plot 1: Explained variance by component
ax1 = axes[0]
n_show = min(20, len(pca_full.explained_variance_ratio_))
ax1.bar(range(1, n_show+1), pca_full.explained_variance_ratio_[:n_show], alpha=0.7, label='Individual')
ax1.plot(range(1, n_show+1), cumsum[:n_show], 'r-o', linewidth=2, label='Cumulative')
ax1.axhline(y=0.95, color='g', linestyle='--', label='95% threshold')
ax1.set_xlabel('Principal Component', fontsize=12)
ax1.set_ylabel('Explained Variance Ratio', fontsize=12)
ax1.set_title('PCA Explained Variance Analysis', fontsize=14, fontweight='bold')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Plot 2: 2D PCA projection
ax2 = axes[1]
pca_2d = PCA(n_components=2)
X_pca_2d = pca_2d.fit_transform(X_scaled)

# Sample for visualization if dataset is large
sample_size = min(5000, len(X_pca_2d))
indices = np.random.choice(len(X_pca_2d), sample_size, replace=False)

scatter = ax2.scatter(X_pca_2d[indices, 0], X_pca_2d[indices, 1], 
                      c=y.iloc[indices].astype('category').cat.codes, 
                      cmap='viridis', alpha=0.5, s=10)
ax2.set_xlabel(f'PC1 ({pca_2d.explained_variance_ratio_[0]*100:.1f}% var)', fontsize=12)
ax2.set_ylabel(f'PC2 ({pca_2d.explained_variance_ratio_[1]*100:.1f}% var)', fontsize=12)
ax2.set_title('2D PCA Projection of Data', fontsize=14, fontweight='bold')
plt.colorbar(scatter, ax=ax2, label='Class')

plt.tight_layout()
plt.savefig('images/02_pca_analysis.png', dpi=150, bbox_inches='tight')
plt.show()
print("\n✅ Figure saved: images/02_pca_analysis.png")

In [ ]:
# Evaluate PCA-transformed features
pca_opt = PCA(n_components=n_components_95)
X_pca = pca_opt.fit_transform(X_scaled)

rf_pca = RandomForestClassifier(n_estimators=50, random_state=42, n_jobs=-1)
scores_pca = cross_val_score(rf_pca, X_pca, y, cv=5, scoring='accuracy')

print(f"\n📈 PCA PERFORMANCE COMPARISON:")
print(f"   Baseline (original {X_scaled.shape[1]} features): {scores_baseline.mean():.4f} (+/- {scores_baseline.std()*2:.4f})")
print(f"   PCA ({n_components_95} components):              {scores_pca.mean():.4f} (+/- {scores_pca.std()*2:.4f})")
print(f"   Accuracy change: {(scores_pca.mean() - scores_baseline.mean())*100:+.2f}%")
print(f"   Feature reduction: {(1 - n_components_95/X_scaled.shape[1])*100:.1f}%")




from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

# LDA for dimensionality reduction (max n_components = n_classes - 1)
lda = LinearDiscriminantAnalysis(n_components=min(len(np.unique(y))-1, 50))
X_lda = lda.fit_transform(X_scaled, y)

rf_lda = RandomForestClassifier(n_estimators=50, random_state=42, n_jobs=-1)
scores_lda = cross_val_score(rf_lda, X_lda, y, cv=5, scoring='accuracy')
print(f"LDA: {scores_lda.mean():.4f} (+/- {scores_lda.std()*2:.4f})")




---

## 4. Feature Selection: Information-Theoretic Approach

**Justification**: Select the most informative features using mutual information, which captures non-linear dependencies that correlation-based methods miss.

In [ ]:
# Feature Selection using Mutual Information
print("=" * 60)
print("FEATURE SELECTION: MUTUAL INFORMATION")
print("=" * 60)

# Calculate mutual information scores
mi_scores = mutual_info_classif(X_scaled, y, random_state=42)
mi_df = pd.DataFrame({'Feature': numeric_cols, 'MI_Score': mi_scores})
mi_df = mi_df.sort_values('MI_Score', ascending=False)

print(f"\n📊 TOP 15 FEATURES BY MUTUAL INFORMATION:")
print(mi_df.head(15).to_string(index=False))

In [ ]:
# # Visualize feature importance
# fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# # Plot 1: Mutual Information scores
# ax1 = axes[0]
# top_n = 15
# top_features = mi_df.head(top_n)
# colors = plt.cm.RdYlGn(np.linspace(0.2, 0.8, top_n))
# bars = ax1.barh(range(top_n), top_features['MI_Score'].values[::-1], color=colors[::-1])
# ax1.set_yticks(range(top_n))
# ax1.set_yticklabels(top_features['Feature'].values[::-1])
# ax1.set_xlabel('Mutual Information Score', fontsize=12)
# ax1.set_title('Top 15 Features by Mutual Information', fontsize=14, fontweight='bold')
# ax1.grid(True, alpha=0.3, axis='x')

# # Plot 2: Feature selection comparison
# ax2 = axes[1]
# k_values = [5, 10, 15, 20, 25, 30]
# k_scores = []
# for k in k_values:
#     top_k_features = mi_df.head(k)['Feature'].tolist()
#     X_selected = X_numeric[top_k_features]
#     X_sel_scaled = StandardScaler().fit_transform(X_selected)
#     rf_sel = RandomForestClassifier(n_estimators=50, random_state=42, n_jobs=-1)
#     scores = cross_val_score(rf_sel, X_sel_scaled, y, cv=3, scoring='accuracy')
#     k_scores.append(scores.mean())

# ax2.plot(k_values, k_scores, 'bo-', linewidth=2, markersize=8)
# ax2.axhline(y=scores_baseline.mean(), color='r', linestyle='--', label='All features baseline')
# ax2.set_xlabel('Number of Selected Features', fontsize=12)
# ax2.set_ylabel('Cross-Validation Accuracy', fontsize=12)
# ax2.set_title('Accuracy vs Number of Features', fontsize=14, fontweight='bold')
# ax2.legend()
# ax2.grid(True, alpha=0.3)

# plt.tight_layout()
# plt.savefig('images/02_feature_importance.png', dpi=150, bbox_inches='tight')
# plt.show()
# print("\n✅ Figure saved: images/02_feature_importance.png")


import matplotlib.pyplot as plt
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

# Visualize feature importance
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# ============================================================================
# Plot 1: Mutual Information scores
# ============================================================================
ax1 = axes[0]
top_n = 5
top_features = mi_df.head(top_n)

# Create colors array matching the number of features
colors = plt.cm.RdYlGn(np.linspace(0.2, 0.8, len(top_features)))

# Create horizontal bar chart
bars = ax1.barh(range(len(top_features)), 
                top_features['MI_Score'].values[::-1], 
                color=colors[::-1])

ax1.set_yticks(range(len(top_features)))
ax1.set_yticklabels(top_features['Feature'].values[::-1], fontsize=10)
ax1.set_xlabel('Mutual Information Score', fontsize=12)
ax1.set_title('Top 5 Features by Mutual Information', fontsize=14, fontweight='bold')
ax1.grid(True, alpha=0.3, axis='x')

# Add value labels on bars
for i, (bar, value) in enumerate(zip(bars, top_features['MI_Score'].values[::-1])):
    ax1.text(value, bar.get_y() + bar.get_height()/2, 
             f'{value:.3f}', 
             ha='left', va='center', fontsize=9, 
             bbox=dict(boxstyle='round,pad=0.3', facecolor='white', alpha=0.7))

# ============================================================================
# Plot 2: Feature selection comparison
# ============================================================================
ax2 = axes[1]
k_values = [5, 10, 15, 20, 25, 30, 40, 50]
k_scores = []

print("\n🔍 Testing different numbers of features...")
for k in k_values:
    # Ensure k doesn't exceed available features
    k_actual = min(k, len(mi_df))
    top_k_features = mi_df.head(k_actual)['Feature'].tolist()
    
    X_selected = X_numeric[top_k_features]
    X_sel_scaled = StandardScaler().fit_transform(X_selected)
    
    rf_sel = RandomForestClassifier(n_estimators=50, random_state=42, n_jobs=-1)
    scores = cross_val_score(rf_sel, X_sel_scaled, y, cv=3, scoring='accuracy')
    k_scores.append(scores.mean())
    print(f"   k={k_actual:2d}: Accuracy = {scores.mean():.4f}")

# Plot the results
ax2.plot(k_values[:len(k_scores)], k_scores, 'bo-', linewidth=2, markersize=8, label='Selected features')
ax2.axhline(y=scores_baseline.mean(), color='r', linestyle='--', linewidth=2, label=f'Baseline ({scores_baseline.mean():.4f})')

# Mark the best point
best_idx = np.argmax(k_scores)
best_k = k_values[best_idx]
best_score = k_scores[best_idx]
ax2.plot(best_k, best_score, 'g*', markersize=20, label=f'Best: k={best_k} ({best_score:.4f})')

ax2.set_xlabel('Number of Selected Features', fontsize=12)
ax2.set_ylabel('Cross-Validation Accuracy', fontsize=12)
ax2.set_title('Accuracy vs Number of Features', fontsize=14, fontweight='bold')
ax2.legend(fontsize=10)
ax2.grid(True, alpha=0.3)

# Add percentage improvement annotations
for i, (k, score) in enumerate(zip(k_values[:len(k_scores)], k_scores)):
    improvement = (score - scores_baseline.mean()) * 100
    if abs(improvement) > 0.5:  # Only show significant changes
        ax2.annotate(f'{improvement:+.1f}%', 
                    xy=(k, score), 
                    xytext=(5, 5), 
                    textcoords='offset points',
                    fontsize=8, 
                    alpha=0.7)

plt.tight_layout()
plt.savefig('images/02_feature_importance.png', dpi=150, bbox_inches='tight')
plt.show()

print("\n✅ Figure saved: images/02_feature_importance.png")
print(f"\n🏆 Optimal number of features: {best_k} (Accuracy: {best_score:.4f})")
print(f"   Improvement over baseline: {(best_score - scores_baseline.mean())*100:+.2f}%")

In [ ]:
# Summary comparison table
print("\n" + "=" * 60)
print("FEATURE ENGINEERING SUMMARY")
print("=" * 60)

summary_data = {
    'Representation': [
        'Original Features',
        'Statistical Features',
        f'PCA ({n_components_95} components)',
        f'Top-{top_n} by MI'
    ],
    'Num Features': [
        len(numeric_cols),
        X_stat.shape[1],
        n_components_95,
        top_n
    ],
    'CV Accuracy': [
        f'{scores_baseline.mean():.4f}',
        f'{scores_stat.mean():.4f}',
        f'{scores_pca.mean():.4f}',
        f'{k_scores[k_values.index(15)]:.4f}' if 15 in k_values else 'N/A'
    ],
    'Std Dev': [
        f'{scores_baseline.std():.4f}',
        f'{scores_stat.std():.4f}',
        f'{scores_pca.std():.4f}',
        'N/A'
    ]
}

summary_df = pd.DataFrame(summary_data)
print("\n")
print(summary_df.to_string(index=False))

In [24]:
# Save engineered features for next notebook
best_features = mi_df.head(20)['Feature'].tolist()
X_engineered = X_stat[best_features + list(stat_features.columns)]

# Save to file
X_engineered.to_pickle('data/engineered_features.pkl')
y.to_pickle('data/target.pkl')

# Update metadata
with open('data/metadata.json', 'r') as f:
    metadata = json.load(f)

metadata['selected_features'] = best_features
metadata['statistical_features'] = list(stat_features.columns)
metadata['pca_components_95'] = str(n_components_95)

with open('data/metadata.json', 'w') as f:
    json.dump(metadata, f, indent=2)

print("\n✅ Engineered features saved to 'data/engineered_features.pkl'")
print("✅ Target variable saved to 'data/target.pkl'")
print("✅ Metadata updated with feature engineering info")


✅ Engineered features saved to 'data/engineered_features.pkl'
✅ Target variable saved to 'data/target.pkl'
✅ Metadata updated with feature engineering info


---

## Summary

### Feature Engineering Techniques Applied:

| Technique | Description | Domain Justification | Impact |
|-----------|-------------|---------------------|--------|
| Statistical Features | Row-wise mean, std, skew, kurtosis, IQR | Captures distribution changes in sensor readings | Improved accuracy |
| PCA Reduction | Extract principal components for 95% variance | Removes redundant information, speeds up training | Minimal accuracy loss, faster training |
| Mutual Information Selection | Select top features by information content | Identifies most predictive features for anomaly detection | Focused model with key features |

### Key Insights:
1. Statistical features enhance model understanding of data distribution
2. PCA achieves significant dimensionality reduction with minimal information loss
3. Feature selection identifies the most informative features for the classification task

**Next >>**: Machine Learning Model Design (Notebook 03)